In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# Inputs
parcels = r".\Inputs\remm_base_year_2019.gdb\parcels_2019"
grid = r".\Inputs\grid.shp" # 15km grid

In [3]:
# output directory
if not os.path.exists('Outputs'): os.makedirs('Outputs') 
outputs = ['.\\Outputs', 'scratch.gdb', 'zoning_surface.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])
if not arcpy.Exists(gdb): 
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])
print('Output gdb path: {}'.format(gdb))
print('Output gdb2 path: {}'.format(gdb2))

Output gdb path: .\Outputs\scratch.gdb
Output gdb2 path: .\Outputs\zoning_surface.gdb


In [4]:
# create grid
area = '.5 SquareKilometers'
grid2 = arcpy.GenerateTessellation_management(os.path.join(os.path.realpath(r'.\Outputs\zoning_surface.gdb'), '_00_grid2'), arcpy.Describe(parcels).extent, "SQUARE", area,)

In [5]:
# copy input parcels
p_copy = arcpy.FeatureClassToFeatureClass_conversion(parcels, gdb,'_00_parcels_copy')
parcels_lyr = arcpy.MakeFeatureLayer_management(p_copy, 'parcels_lyr')
print(arcpy.GetCount_management(parcels_lyr))

query = """ building_sqft IS NULL """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
arcpy.DeleteFeatures_management(parcels_lyr)
print(arcpy.GetCount_management(parcels_lyr))

# add and calculate parcel FAR attribute
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')
arcpy.AddField_management(parcels_lyr, 'FAR', "FLOAT")
arcpy.CalculateField_management(parcels_lyr, 'FAR', '''!building_sqft! / !parcel_acres!''' )

711978
569055


<Result 'parcels_lyr'>

## Median Year Built by Building Type

In [6]:
query = """ building_type_id = 1 """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid2
join_features = parcels_lyr
output_features = os.path.join(gdb, "_01_year_built_sf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('year_built')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj5 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

499730


<Result 'parcels_lyr'>

In [7]:
query = """ building_type_id = 2 """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid2
join_features = parcels_lyr
output_features = os.path.join(gdb, "_01_year_built_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('year_built')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj6 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

26530


<Result 'parcels_lyr'>

In [8]:
query = """ building_type_id = 3 """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid2
join_features = parcels_lyr
output_features = os.path.join(gdb, "_01_year_built_ind_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('year_built')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj7 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

9092


<Result 'parcels_lyr'>

In [9]:
query = """ building_type_id in (4,5,6,7,8,9,10,11,13) """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid2
join_features = parcels_lyr
output_features = os.path.join(gdb, "_01_year_built_nobind_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('year_built')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj8 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

23850


<Result 'parcels_lyr'>

## Median MAXDUA

In [10]:
target_features = grid2
join_features = parcels
output_features = os.path.join(gdb, "_02_grid_parcels_maxdua_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('max_dua')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

## Median MAXFAR by Building Type

In [11]:
query = """ building_type_id = 1 """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid
join_features = parcels_lyr
output_features = os.path.join(gdb, "_03_grid_parcels_sf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('FAR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj1 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

499730


<Result 'parcels_lyr'>

In [12]:
query = """ building_type_id = 2 """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid
join_features = parcels_lyr
output_features = os.path.join(gdb, "_03_grid_parcels_mf_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('FAR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

26530


<Result 'parcels_lyr'>

In [13]:
query = """ building_type_id = 3 """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION', query)
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid
join_features = parcels_lyr
output_features = os.path.join(gdb, "_03_grid_parcels_ind_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('FAR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj3 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

9092


<Result 'parcels_lyr'>

In [14]:
query = """ building_type_id in (4,5,6,7,8,9,10,11,13) """
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'NEW_SELECTION')
print(arcpy.GetCount_management(parcels_lyr))

target_features = grid
join_features = parcels_lyr
output_features = os.path.join(gdb, "_03_grid_parcels_nonind_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('FAR')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Median'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj4 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION')

569055


<Result 'parcels_lyr'>

In [15]:
grid_sdf = pd.DataFrame.spatial.from_featureclass(grid)
grid2_sdf = pd.DataFrame.spatial.from_featureclass(grid2[0])

sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])[['GRID_ID', 'max_dua']].copy()
sj_df.columns = ['GRID_ID', 'MAXDUA_MED']

sj1_df = pd.DataFrame.spatial.from_featureclass(sj1[0])[['GRID_ID', 'FAR']].copy()
sj1_df.columns = ['GRID_ID', 'FAR_SF']

sj2_df = pd.DataFrame.spatial.from_featureclass(sj2[0])[['GRID_ID', 'FAR']].copy()
sj2_df.columns = ['GRID_ID', 'FAR_MF']

sj3_df = pd.DataFrame.spatial.from_featureclass(sj3[0])[['GRID_ID', 'FAR']].copy()
sj3_df.columns = ['GRID_ID', 'FAR_IND']

sj4_df = pd.DataFrame.spatial.from_featureclass(sj4[0])[['GRID_ID', 'FAR']].copy()
sj4_df.columns = ['GRID_ID', 'FAR_NONIND']

sj5_df = pd.DataFrame.spatial.from_featureclass(sj5[0])[['GRID_ID', 'year_built']].copy()
sj5_df.columns = ['GRID_ID', 'YB_SF']
sj5_df['YB_SF'] = sj5_df['YB_SF'].fillna(0).round(0).astype(int)

sj6_df = pd.DataFrame.spatial.from_featureclass(sj6[0])[['GRID_ID', 'year_built']].copy()
sj6_df.columns = ['GRID_ID', 'YB_MF']
sj6_df['YB_MF'] = sj6_df['YB_MF'].fillna(0).round(0).astype(int)

sj7_df = pd.DataFrame.spatial.from_featureclass(sj7[0])[['GRID_ID', 'year_built']].copy()
sj7_df.columns = ['GRID_ID', 'YB_IND']
sj7_df['YB_IND'] = sj7_df['YB_IND'].fillna(0).round(0).astype(int)

sj8_df = pd.DataFrame.spatial.from_featureclass(sj8[0])[['GRID_ID', 'year_built']].copy()
sj8_df.columns = ['GRID_ID', 'YB_NONIND']
sj8_df['YB_NONIND'] = sj8_df['YB_NONIND'].fillna(0).round(0).astype(int)

In [16]:
# max far
merged = grid_sdf.merge(sj1_df, left_on='GRID_ID', right_on='GRID_ID', how='left')
merged = merged.merge(sj2_df, left_on='GRID_ID', right_on='GRID_ID', how='left')
merged = merged.merge(sj3_df, left_on='GRID_ID', right_on='GRID_ID', how='left')
merged = merged.merge(sj4_df, left_on='GRID_ID', right_on='GRID_ID', how='left')

# everything else
merged2 = grid2_sdf.merge(sj_df, left_on='GRID_ID', right_on='GRID_ID', how='left')
merged2 = merged2.merge(sj5_df, left_on='GRID_ID', right_on='GRID_ID', how='left')
merged2 = merged2.merge(sj6_df, left_on='GRID_ID', right_on='GRID_ID', how='left')
merged2 = merged2.merge(sj7_df, left_on='GRID_ID', right_on='GRID_ID', how='left')
merged2 = merged2.merge(sj8_df, left_on='GRID_ID', right_on='GRID_ID', how='left')

In [17]:
print(merged.shape)
print(merged2.shape)

(338, 9)
(19404, 8)


In [18]:
result = os.path.join(gdb2, 'base_year_surface_far')
merged.spatial.to_featureclass(location=result, sanitize_columns=False)

'e:\\Projects\\Create_Zoning_Surface_For_REMM\\Outputs\\zoning_surface.gdb\\base_year_surface_far'

In [19]:
result2 = os.path.join(gdb2, 'base_year_surface_maxdua_year_built')
merged2.spatial.to_featureclass(location=result2, sanitize_columns=False)

'e:\\Projects\\Create_Zoning_Surface_For_REMM\\Outputs\\zoning_surface.gdb\\base_year_surface_maxdua_year_built'

In [20]:
# os.startfile(r".\Create_Base_Year_Surface_.aprx")